In [1]:
import pandas as pd

In [2]:
csv_file = 'restaurants_info.csv'

try:
    # Read CSV file with error-handling
    csv_df = pd.read_csv(csv_file, delimiter=';')
    print(csv_df.head)
    num_records = len(csv_df)
    print("Number of records:", num_records)
except pd.errors.ParserError as e:
    print(f"My Error: Error parsing CSV file: {e}")

json_file = 'restaurants_list.json'

try:
    # Read JSON file with error-handling
    json_df = pd.read_json(json_file)
    print(json_df.head())
    num_records = len(json_df)
    print("Number of records:", num_records)
except pd.errors.ParserError as e:
    print(f"My Error: Error parsing JSON file: {e}")

<bound method NDFrame.head of       objectID              food_type  stars_count  reviews_count  \
0       116272                  Steak          4.2            204   
1       138901                  Sushi          4.3             75   
2        96892                Italian          4.3            161   
3       108532               American          4.5            159   
4        25840                  Sushi          4.5            332   
...        ...                    ...          ...            ...   
4995     10363  Contemporary American          4.4           1120   
4996      2131                Italian          4.4           1273   
4997       763            Californian          4.3           2874   
4998      2177                  Asian          3.9           1062   
4999       963                Seafood          4.3           2451   

          neighborhood    phone_number    price_range    dining_style  
0          Pepper Pike  (216) 378-8988     $31 to $50     Fine Dining

In [7]:
# Merge the DataFrames on the "objectID" column
merged_df = pd.merge(json_df, csv_df, on="objectID", how="inner")

# Display the merged DataFrame
# print(merged_df.head())

columns = merged_df.columns
print("Columns:", columns)

# Add a UK restaurant to test location precision 
new_record = {
    'objectID': 99999,
    'name': 'Pizzucci',
    'address': '5, The Promenade, Gloucester Road',
    'area': 'Bishopston',
    'city': 'Bristol',
    'country': 'United Kingdom',
    'image_url': 'pizzucci.png',
    'mobile_reserve_url': 'http://pizzucci.co.uk/',
    'payment_options': 'Visa, MasterCard',
    'phone': '01173790908',
    'postal_code': 'BS7 8AL',
    'price': 25.00,
    'reserve_url': 'http://pizzucci.co.uk/',
    'state': 'UK',
    '_geoloc': {'lat': 51.471988, 'lng': -2.59229},
    'food_type': 'Italian',
    'stars_count': 4.5,
    'reviews_count': 130,
    'neighborhood': 'Gloucester Road',
    'phone_number': '01173790908',
    'price_range': '$30 and under',
    'dining_style': 'Casual Dining'
}

# Add the new row to the DataFrame
merged_df.loc[len(merged_df.index)] = new_record

# Specify the path for the new CSV file
output_csv_file = 'merged_restaurant_data.csv'

# Write the merged DataFrame to a new CSV file (could import it manually into Algolia by file)
merged_df.to_csv(output_csv_file, index=False)
num_records = len(merged_df)
# print("Number of records:", num_records)

# print(f"Merged data has been written to '{output_csv_file}'.")

Columns: Index(['objectID', 'name', 'address', 'area', 'city', 'country', 'image_url',
       'mobile_reserve_url', 'payment_options', 'phone', 'postal_code',
       'price', 'reserve_url', 'state', '_geoloc', 'food_type', 'stars_count',
       'reviews_count', 'neighborhood', 'phone_number', 'price_range',
       'dining_style'],
      dtype='object')


In [8]:
print(merged_df.last)

<bound method NDFrame.last of       objectID                          name  \
0       101422                          Town   
1       113494                Plates Kitchen   
2       152470                 Pax Americana   
3       145693  Vinotopia Restaurant and Bar   
4        22588              Biagio's Osteria   
...        ...                           ...   
4996    148597             Dylan's Candy Bar   
4997     56539      Dexters of Thornton Park   
4998    118219                Los Americanos   
4999     25840                          SASA   
5000     99999                      Pizzucci   

                                address                            area  \
0                       348 Main Street               Denver / Colorado   
1                      301 Glenwood Ave  Raleigh / Durham / Chapel Hill   
2                   4319 Montrose Blvd.                         Houston   
3               5724 West 136th Terrace                     Kansas City   
4                 

In [9]:
# Let's ping over the DF through the Algolia API 💪

from algoliasearch.search_client import SearchClient

# Initialize the Algolia client
client = SearchClient.create('GK7Q3EEC39', '22ba65d4d17fc040d760e0ffa2a58ef4')
index = client.init_index('FeastFinder')

# Convert DataFrame to JSON
records = merged_df.to_dict(orient='records')

# Delete all records from the index
response = index.clear_objects()

# Handle the response
if 'error' in response:
    print(f"Error clearing index: {response['message']}")
else:
    print("Index cleared successfully.")

# Import data to Algolia
response = index.save_objects(records)

# Handle the response
if 'error' in response:
    print(f"Error importing data: {response['message']}")
else:
    print("Data imported successfully.")

Index cleared successfully.
Data imported successfully.
